# 1. Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_gender = pd.read_csv('dataset_WWW2019/dataset_WWW_Checkins_anonymized.txt', sep = '\t', header = None)
#df_friends = pd.read_csv('dataset_WWW2019/dataset_WWW_friendship_new.txt', sep = '\t', header = None)
df_poi = pd.read_csv('dataset_WWW2019/raw_Checkins_anonymized.txt', sep = '\t', header = None)

In [6]:
df_checkin2 = df_checkin.rename(columns={0:'User_ID' , 1: 'Venue_ID', 2:'UTC_Time', 3:'Timezone_Offset'})

In [10]:
df_poi = pd.read_csv('C:/Users/piai/Desktop/code_file/dataset_WWW2019/raw_POIs.txt',sep = '\t',header=None, encoding= 'utf-8')

In [11]:
df_poi.head(10)

,0,1,2,3,4
0,3fd66200f964a52000e61ee3,40.729209,-73.998753,Post Office,US
1,3fd66200f964a52000e71ee3,40.733596,-74.003139,Jazz Club,US
2,3fd66200f964a52000e81ee3,40.758102,-73.975734,Gym,US
3,3fd66200f964a52000ea1ee3,40.732456,-74.003755,Indian Restaurant,US
4,3fd66200f964a52000ec1ee3,42.345907,-71.087001,Indian Restaurant,US
5,3fd66200f964a52000ee1ee3,39.933178,-75.159262,Sandwich Place,US
6,3fd66200f964a52000ef1ee3,34.188944,-118.320329,Karaoke Bar,US
7,3fd66200f964a52000f01ee3,34.010279,-118.495523,Beer Garden,US
8,3fd66200f964a52000f11ee3,40.652766,-74.003092,Bowling Alley,US
9,3fd66200f964a52001e51ee3,40.726961,-73.980039,Dive Bar,US


In [12]:
df_poi = df_poi.rename(columns={0:'Venue_ID' , 1: 'Lat', 2:'Long', 3:'Category', 4:'Country_Code'})

In [13]:
df = df_checkin2.merge(df_poi, on='Venue_ID', how='left')

In [14]:
df.head()

,User_ID,Venue_ID,UTC_Time,Timezone_Offset,Lat,Long,Category,Country_Code
0,546830,4f5e3a72e4b053fd6a4313f6,Tue Apr 03 18:00:06 +0000 2012,240.0,55.696132,37.557842,Comedy Club,RU
1,822121,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,180.0,41.029717,28.974420,Bar,TR
2,2277773,4a85b1b3f964a520eefe1fe3,Tue Apr 03 18:00:08 +0000 2012,-240.0,40.748954,-73.992257,Coffee Shop,US
3,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300.0,30.270786,-97.753153,Office,US
4,1139878,4d9254ef62ad5481fa6e6a4b,Tue Apr 03 18:00:08 +0000 2012,-180.0,-29.169531,-51.179939,School,BR


In [15]:
df.to_csv("gender-checkins-complete.csv",sep = '\t')

# 메모리 기반 협업 필터링을 통한 User의 다음 체크인 장소 예측 알고리즘 구현

In [16]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [34]:
# Check-In Data
check_ins = pd.read_csv('gender-checkins-complete.csv', delimiter='\t', index_col=0)

In [35]:
check_ins.head()

,User_ID,Venue_ID,UTC_Time,Timezone_Offset,Lat,Long,Category,Country_Code
0,546830,4f5e3a72e4b053fd6a4313f6,Tue Apr 03 18:00:06 +0000 2012,240.0,55.696132,37.557842,Comedy Club,RU
1,822121,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,180.0,41.029717,28.974420,Bar,TR
2,2277773,4a85b1b3f964a520eefe1fe3,Tue Apr 03 18:00:08 +0000 2012,-240.0,40.748954,-73.992257,Coffee Shop,US
3,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300.0,30.270786,-97.753153,Office,US
4,1139878,4d9254ef62ad5481fa6e6a4b,Tue Apr 03 18:00:08 +0000 2012,-180.0,-29.169531,-51.179939,School,BR


## Preprocessing

In [36]:
threshold = 80 # 스레스홀드 미만인 체크인 기록 보유 user는 제거하고 보도록.

value_counts = check_ins['User_ID'].value_counts() # Specific column 
to_remove = value_counts[value_counts <= threshold].index
check_ins = check_ins[~check_ins['User_ID'].isin(to_remove)]

In [37]:
"There are %d check-ins, made by %d users, in %d venues." %(len(check_ins)
, check_ins['User_ID'].nunique(), check_ins['Venue_ID'].nunique())

'There are 12438445 check-ins, made by 77635 users, in 2572190 venues.'

In [39]:
#split into train/test
split = 0.6

X_train = (check_ins.groupby('User_ID',group_keys=False)
        .apply(lambda x: x.nlargest(int(len(x) * split), 'User_ID')))
X_train

,User_ID,Venue_ID,UTC_Time,Timezone_Offset,Lat,Long,Category,Country_Code
5673346,15,4bb14a0cf964a520e9893ce3,Wed Apr 25 21:12:50 +0000 2012,-420.0,36.292377,-119.325095,General College & University,US
5678957,15,4c05a640d3842d7fe488be41,Wed Apr 25 21:56:17 +0000 2012,-420.0,36.323795,-119.348035,Gym,US
5706331,15,4b4cfad1f964a5206ec626e3,Wed Apr 25 23:56:45 +0000 2012,-420.0,36.312187,-119.313529,Caf��,US
6028531,15,4bb14a0cf964a520e9893ce3,Fri Apr 27 15:16:23 +0000 2012,-420.0,36.292377,-119.325095,General College & University,US
6028679,15,4daf9aa9a86e9639ef511717,Fri Apr 27 15:16:54 +0000 2012,-420.0,36.292787,-119.325866,Field,US
...,...,...,...,...,...,...,...,...
25859047,2456374,4c1decfeb306c928004d66b7,Sun Aug 12 17:48:16 +0000 2012,480.0,-0.496481,117.145715,Coffee Shop,ID
25859122,2456374,4be7ecddee96c928af6cfdbf,Sun Aug 12 17:48:45 +0000 2012,480.0,-0.497107,117.145177,Hotel,ID
25956637,2456374,4bf5d6bbc07c9c74034abbef,Mon Aug 13 14:21:06 +0000 2012,480.0,-0.495215,117.143945,Police Station,ID
25957843,2456374,4c7aeaf3794e224b492e6a28,Mon Aug 13 14:27:24 +0000 2012,480.0,-0.487858,117.147607,High School,ID


In [40]:
X_test = check_ins[~check_ins.isin(X_train)].dropna()

In [41]:
X_train_counts = X_train.groupby(['User_ID','Venue_ID']).size().reset_index(name="Frequency")

In [42]:
Total_Visits = X_train.groupby(['User_ID']).size().reset_index(name="Total_Visits")
X_train_counts = pd.merge(X_train_counts, Total_Visits, on = 'User_ID', how='left', sort = 'False')
del X_train

In [43]:
np.mean(Total_Visits['Total_Visits'])

95.72665679139563

In [44]:
X_train_counts['Adj_Freq'] = X_train_counts['Frequency'] / X_train_counts['Total_Visits'] * 1.0

In [45]:
X_train_counts['Adj_Freq']

0          0.009009
1          0.009009
2          0.018018
3          0.009009
4          0.009009
             ...   
3518606    0.015152
3518607    0.015152
3518608    0.015152
3518609    0.015152
3518610    0.015152
Name: Adj_Freq, Length: 3518611, dtype: float64

In [46]:
X_train_counts.head()

,User_ID,Venue_ID,Frequency,Total_Visits,Adj_Freq
0,15,4a9700d4f964a520e92720e3,1,111,0.009009
1,15,4af91213f964a520ed1022e3,1,111,0.009009
2,15,4b12001af964a520068823e3,2,111,0.018018
3,15,4b220ceef964a5206d4324e3,1,111,0.009009
4,15,4b43bbbaf964a520e9e725e3,1,111,0.009009


In [47]:
X_train_counts = X_train_counts.pivot(index='User_ID', columns='Venue_ID', values='Adj_Freq')

c:\Users\piai\anaconda3\lib\site-packages\pandas\core\reshape\reshape.py:130: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [ ]:
X_train_counts = X_train_counts.fillna(0)